In [1]:
import torch
import transformers

from transformers import BertTokenizer
from transformers import BertForQuestionAnswering

## Preprocessing

In [2]:
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

c:\Users\Asus\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Asus\.cache\huggingface\hub\models--bert-large-uncased-whole-word-masking-finetuned-squad. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

In [15]:
with open("wiki_article.txt", "r", encoding = 'utf-8') as file:
    text = file.read()

In [16]:
# text_document = "" 

Encoding error as token length exceeded

In [17]:
# input_ids = tokenizer.encode(text, add_special_tokens = True, return_tensors = 'pt')

Breaking text into parts and then combining the answers

In [18]:
max_chunk_len = 512

In [19]:
chunks = [text[i:i + max_chunk_len] for i in range(0, len(text), max_chunk_len)]

In [42]:
chunks

['The Saturn V was a rocket NASA built to send people to the moon. (The V in the name is the Roman numeral five.) The Saturn V was a type of rocket called a Heavy Lift Vehicle. That means it was very powerful. It was the most powerful rocket that had ever flown successfully. The Saturn V was used in the Apollo program in the 1960s and 1970s. It also was used to launch the Skylab space station.\n\nThe Saturn V rocket was 111 meters (363 feet) tall, about the height of a 36-story-tall building, and 18 meters (60 ',
 'feet) taller than the Statue of Liberty. Fully fueled for liftoff, the Saturn V weighed 2.8 million kilograms (6.2 million pounds), the weight of about 400 elephants. The rocket generated 34.5 million newtons (7.6 million pounds) of thrust at launch, creating more power than 85 Hoover Dams. A car that gets 48 kilometers (30 miles) to the gallon could drive around the world around 800 times with the amount of fuel the Saturn V used for a lunar landing mission. It could launch

In [20]:
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [21]:
model.eval()

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-23): 24 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), ep

In [22]:
answers = []

RuntimeError: The size of tensor a (122) must match the size of tensor b (8) at non-singleton dimension 3

In [23]:
# for chunk in chunks:
#     input_ids = tokenizer.encode(chunk, add_special_tokens=True, return_tensors='pt')

#     question = "What is the length of Saturn V?"
#     input_ids_question = tokenizer.encode(question, add_special_tokens=False, return_tensors='pt')

#     start_scores, end_scores = model(input_ids, input_ids_question)
#     answer_start = torch.argmax(start_scores)
#     answer_end = torch.argmax(end_scores)

#     answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[0][answer_start:answer_end+1]))
#     answers.append(answer)

TypeError: argmax(): argument 'input' (position 1) must be Tensor, not str

In [24]:
# for chunk in chunks:
#     # Tokenize the chunk
#     tokenized_chunk = tokenizer.encode_plus(chunk, add_special_tokens=True, return_tensors='pt', max_length=max_chunk_len, truncation=True)
#     input_ids_chunk = tokenized_chunk['input_ids']
#     token_type_ids_chunk = tokenized_chunk['token_type_ids']
#     attention_mask_chunk = tokenized_chunk['attention_mask']

#     # Run question answering
#     question = "What is the length of Saturn V?"
#     input_ids_question = tokenizer.encode(question, add_special_tokens=False, return_tensors='pt')

#     start_scores, end_scores = model(input_ids_chunk, token_type_ids=token_type_ids_chunk, attention_mask=attention_mask_chunk, start_positions=None, end_positions=None, inputs_embeds=None)
    
#     answer_start = torch.argmax(start_scores)
#     answer_end = torch.argmax(end_scores)
    
#     # Get the answer tokens
#     answer_tokens = input_ids_chunk[0][answer_start:answer_end+1]
#     answer = tokenizer.decode(answer_tokens, skip_special_tokens=True)
    
#     answers.append(answer)

In [38]:
for chunk in chunks:
    # Tokenize the chunk
    tokenized_chunk = tokenizer.encode_plus(chunk, add_special_tokens=True, return_tensors='pt', max_length=max_chunk_len, truncation=True)
    input_ids_chunk = tokenized_chunk['input_ids']
    token_type_ids_chunk = tokenized_chunk['token_type_ids']
    attention_mask_chunk = tokenized_chunk['attention_mask']

    # Run question answering
    question = "What is the length of Saturn V?"
    input_ids_question = tokenizer.encode(question, add_special_tokens=False, return_tensors='pt')

    with torch.no_grad():  # Disable gradient calculation
        outputs = model(input_ids_chunk, token_type_ids=token_type_ids_chunk, attention_mask=attention_mask_chunk)
        start_scores = outputs.start_logits
        end_scores = outputs.end_logits

    # Ensure tensors are on CPU for argmax
    start_scores = start_scores.cpu()
    end_scores = end_scores.cpu()

    answer_start = torch.argmax(start_scores)
    answer_end = torch.argmax(end_scores)

    # Get the answer tokens
    answer_tokens = input_ids_chunk[0][answer_start:answer_end+1]
    answer = tokenizer.decode(answer_tokens, skip_special_tokens=True)

    answers.append(answer)

In [39]:
combined_answer = " ".join(answers)
print("BOSS, the answer is ready!")

BOSS, the answer is ready!


In [40]:
print("Question:", question)
print("Combined Answer:", combined_answer)

Question: What is the length of Saturn V?
Combined Answer: the saturn v was a rocket nasa built to send people to the moon. ( the v in the name is the roman numeral five. ) the saturn v was a type of rocket called a heavy lift vehicle  43, 500 kilograms apollo 6 apollo 11  third stage placed the apollo spacecraft into earth orbit and pushed it toward the moon


In [28]:
# tokenized_text = tokenizer.tokenize(text_document)

In [29]:
# input_ids = tokenizer.convert_tokens_to_ids(tokenized_text)

In [30]:
# input_ids = [tokenizer.cls_token_id] + input_ids + [tokenizer.sep_token_id]

## Load RAW BERT

In [31]:
# model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [32]:
# Setting evaluation mode
# model.eval()

## Running Question Answering

In [33]:
# question = "What is the question"

In [34]:
# input_ids_question = tokenizer.encode(question, add_special_tokens = False, return_tensors = 'pt')

In [35]:
# Get the start and end scores for each token
# start_scores, end_scores = model(input_ids, input_ids_question)

In [36]:
# # Find tokens with max match
# answer_start = torch.argmax(start_scores)
# answer_end = torch.argmax(end_scores)

In [37]:
# COmbine tokens into final answer
# answer = ' '.join(tokenizer.convert_ids_to_tokens(input_ids[0][answer_start:answer_end+1]))